In [1]:
import pandas as pd
import sqlite3
import json
import matplotlib.pyplot as plt

In [2]:
sales = pd.read_csv("sales_cleaned.csv")
agents = pd.read_json("agents_cleaned.json")
buyers = pd.read_json("buyers_cleaned.json")
listings = pd.read_json("listings_final_expanded.json")
property_attributes = pd.read_json("property_attributes_final_expanded.json")


In [3]:
buyers.loc[buyers['loan_taken'] == False, 'loan_provider'] = 'No Loan'

In [21]:
buyers.head()

,buyer_id,sale_id,buyer_type,payment_mode,loan_taken,loan_provider,loan_amount
0,1,L01179,End User,Cash,False,No Loan,0
1,2,L00866,Investor,Cheque,False,No Loan,0
2,3,L00102,Investor,Cheque,True,Axis,2317757
3,4,L00440,Investor,Bank Transfer,False,No Loan,0
4,5,L00059,Investor,UPI,True,HDFC,4191221


In [4]:
import sqlite3

conn = sqlite3.connect("real_estate_database1.sqlite")  # Creates a local database file
cursor = conn.cursor()
print("SQLite connection established!")


SQLite connection established!


In [5]:
create_listings_table = """
CREATE TABLE IF NOT EXISTS listings (
    Listing_ID INTEGER PRIMARY KEY,
    City TEXT,
    Property_Type TEXT,
    Price REAL,
    Sqft REAL,
    Agent_ID INTEGER,
    Date_Listed DATE,
    Latitude REAL,
    Longitude REAL
);
"""
cursor.execute(create_listings_table)
conn.commit()

In [6]:
cursor.execute("PRAGMA table_info(listings);")
columns = cursor.fetchall()

for col in columns:
    print(col)

(0, 'Listing_ID', 'TEXT', 0, None, 0)
(1, 'City', 'TEXT', 0, None, 0)
(2, 'Property_Type', 'TEXT', 0, None, 0)
(3, 'Price', 'REAL', 0, None, 0)
(4, 'Sqft', 'REAL', 0, None, 0)
(5, 'Date_Listed', 'TEXT', 0, None, 0)
(6, 'Agent_ID', 'TEXT', 0, None, 0)
(7, 'Latitude', 'REAL', 0, None, 0)
(8, 'Longitude', 'REAL', 0, None, 0)


In [8]:
create_property_attributes_table = """
CREATE TABLE IF NOT EXISTS property_attributes (
    attribute_id INTEGER PRIMARY KEY,
    listing_id INTEGER,
    bedrooms INTEGER,
    bathrooms INTEGER,
    floor_number INTEGER,
    total_floors INTEGER,
    year_built INTEGER,
    is_rented BOOLEAN,
    tenant_count INTEGER,
    furnishing_status TEXT,
    metro_distance_km REAL,
    parking_available BOOLEAN,
    power_backup BOOLEAN,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id)
);
"""
cursor.execute(create_property_attributes_table)
conn.commit()

In [9]:
create_agents_table = """
CREATE TABLE IF NOT EXISTS agents (
    Agent_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Phone TEXT,
    Email TEXT,
    commission_rate REAL,
    deals_closed INTEGER,
    rating REAL,
    experience_years INTEGER,
    avg_closing_days  INTEGER
);
"""
cursor.execute(create_agents_table)
conn.commit()

In [10]:
create_buyers_table = """
CREATE TABLE IF NOT EXISTS buyers (
    buyer_id INTEGER PRIMARY KEY,
    sale_id INTEGER,
    buyer_type TEXT,
    payment_mode TEXT,
    loan_taken BOOLEAN,
    loan_provider TEXT,
    loan_amount REAL,
    FOREIGN KEY (sale_id) REFERENCES sales(sale_id)
);
"""

cursor.execute(create_buyers_table)
conn.commit()

In [11]:
cursor.execute("drop table sales;")
conn.commit()

In [12]:
create_sales_table = """
CREATE TABLE IF NOT EXISTS sales (
    Listing_ID INTEGER,
    Sale_Price REAL,
    Date_Sold DATE,
    Days_on_Market REAL
);
"""

In [13]:
listings.to_sql("listings", conn, if_exists="replace", index=False)
property_attributes.to_sql("property_attributes", conn, if_exists="replace", index=False)
agents.to_sql("agents", conn, if_exists="replace", index=False)
sales.to_sql("sales", conn, if_exists="replace", index=False)
buyers.to_sql("buyers", conn, if_exists="replace", index=False)

20000

In [14]:
from tabulate import tabulate

In [15]:
#What is the average listing price by city? 

query1 = "select City ,Avg(price) from listings group by city; "
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = ["City", "listing price"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+-------------+-----------------+
| City        |   listing price |
+=============+=================+
| Chicago     |     2.43068e+06 |
+-------------+-----------------+
| Houston     |     2.43681e+06 |
+-------------+-----------------+
| Los Angeles |     2.44242e+06 |
+-------------+-----------------+
| New York    |     2.49332e+06 |
+-------------+-----------------+
| Phoenix     |     2.45996e+06 |
+-------------+-----------------+


In [16]:
#What is the average price per square foot by property type?

query2 = "select property_type, Avg(price/sqft) as Avg_Price_Per_Sqft from listings group by property_type; "
cursor.execute(query2)
result2 = cursor.fetchall()

# Define table headers
headers = ["property_type", "Avg_Price_Per_Sqft"]

# Print the result as a table
print(tabulate(result2, headers=headers, tablefmt="grid"))

+-----------------+----------------------+
| property_type   |   Avg_Price_Per_Sqft |
+=================+======================+
| Apartment       |              792.203 |
+-----------------+----------------------+
| Condo           |              754.676 |
+-----------------+----------------------+
| House           |              796.041 |
+-----------------+----------------------+
| Townhouse       |              789.745 |
+-----------------+----------------------+


In [17]:
#how does furnishing status impact property prices?
#query3 = " select avg(l.price) as Avg_price ,furnishing_status from listings l inner join property_attributes p ON l.listing_id = p.listing_id group by furnishing_status;"
query3 = """
SELECT
    p.furnishing_status,
    COUNT(*) AS total_listings,
    AVG(l.price) AS avg_price,
    AVG(l.price / l.sqft) AS avg_price_per_sqft
FROM listings l
JOIN property_attributes p
    ON l.listing_id = p.listing_id
WHERE l.sqft > 0
GROUP BY p.furnishing_status
ORDER BY avg_price_per_sqft DESC;
"""

cursor.execute(query3)
result3 = cursor.fetchall()

# Define table headers
headers = ["furnishing_status","total_listings", "avg_price","avg_price_per_sqft"]

# Print the result as a table
print(tabulate(result3, headers=headers, tablefmt="grid"))

+---------------------+------------------+-------------+----------------------+
| furnishing_status   |   total_listings |   avg_price |   avg_price_per_sqft |
+=====================+==================+=============+======================+
| Furnished           |             6977 | 2.46354e+06 |              792.408 |
+---------------------+------------------+-------------+----------------------+
| Semi-Furnished      |             7013 | 2.46119e+06 |              780.408 |
+---------------------+------------------+-------------+----------------------+
| Unfurnished         |             7210 | 2.43398e+06 |              776.638 |
+---------------------+------------------+-------------+----------------------+


In [19]:
#Do properties closer to metro stations command higher prices?

query4 = """
SELECT
    CASE
        WHEN p.metro_distance_km <= 0.5 THEN '0–0.5 km'
        WHEN p.metro_distance_km <= 1 THEN '0.5–1 km'
        WHEN p.metro_distance_km <= 2 THEN '1–2 km'
        WHEN p.metro_distance_km <= 3 THEN '2–3 km'
        WHEN p.metro_distance_km <= 5 THEN '3–5 km'
        ELSE '5+ km'
    END AS metro_distance_band,
    COUNT(*) AS total_listings,
    AVG(l.price) AS avg_price,
    AVG(l.price / l.sqft) AS avg_price_per_sqft
FROM listings l
JOIN property_attributes p
    ON l.listing_id = p.listing_id
WHERE l.sqft > 0
GROUP BY metro_distance_band
ORDER BY
    MIN(p.metro_distance_km);

"""

cursor.execute(query4)
result4 = cursor.fetchall()

# Define table headers
headers = ["metro_distance_km","total_listings", "avg_price","avg_price_per_sqft"]

# Print the result as a table
print(tabulate(result4, headers=headers, tablefmt="grid"))


+---------------------+------------------+-------------+----------------------+
| metro_distance_km   |   total_listings |   avg_price |   avg_price_per_sqft |
+=====================+==================+=============+======================+
| 0–0.5 km            |              611 | 2.42453e+06 |              772.736 |
+---------------------+------------------+-------------+----------------------+
| 0.5–1 km            |              790 | 2.3577e+06  |              758.004 |
+---------------------+------------------+-------------+----------------------+
| 1–2 km              |             1507 | 2.36944e+06 |              763.437 |
+---------------------+------------------+-------------+----------------------+
| 2–3 km              |             1462 | 2.42991e+06 |              745.946 |
+---------------------+------------------+-------------+----------------------+
| 3–5 km              |             2954 | 2.3924e+06  |              770.492 |
+---------------------+-----------------

In [21]:
query4_1 = """
SELECT
    CASE
        WHEN pa.Metro_Distance_Km <= 1 THEN '0–1 km'
        WHEN pa.Metro_Distance_Km <= 3 THEN '1–3 km'
        WHEN pa.Metro_Distance_Km <= 5 THEN '3–5 km'
        ELSE '5+ km'
    END AS metro_distance_bucket,
    COUNT(*) AS listings,
    ROUND(AVG(l.Price), 2) AS avg_price,
    ROUND(AVG(l.Price * 1.0 / l.sqft), 2) AS avg_price_per_sqft
FROM listings l
JOIN property_attributes pa
  ON l.Listing_ID = pa.Listing_ID
GROUP BY metro_distance_bucket
ORDER BY MIN(pa.Metro_Distance_Km);
"""
cursor.execute(query4_1)
result4_1 = cursor.fetchall()

# Define table headers
headers = ["metro_distance_km","total_listings", "avg_price","avg_price_per_sqft"]

# Print the result as a table
print(tabulate(result4_1, headers=headers, tablefmt="grid"))


+---------------------+------------------+-------------+----------------------+
| metro_distance_km   |   total_listings |   avg_price |   avg_price_per_sqft |
+=====================+==================+=============+======================+
| 0–1 km              |             1401 | 2.38685e+06 |               764.43 |
+---------------------+------------------+-------------+----------------------+
| 1–3 km              |             2969 | 2.39921e+06 |               754.82 |
+---------------------+------------------+-------------+----------------------+
| 3–5 km              |             2954 | 2.3924e+06  |               770.49 |
+---------------------+------------------+-------------+----------------------+
| 5+ km               |            13876 | 2.48364e+06 |               793.68 |
+---------------------+------------------+-------------+----------------------+


In [22]:
#Are rented properties priced differently from non-rented ones?
### 
query5 = """
SELECT
    p.is_rented as RentedorNot,
    COUNT(*) AS total_listings,
    AVG(l.price) AS avg_price,
    AVG(l.price / l.sqft) AS avg_price_per_sqft
FROM listings l
JOIN property_attributes p
    ON l.listing_id = p.listing_id
WHERE l.sqft > 0
GROUP BY p.is_rented ;
"""

cursor.execute(query5)
result5 = cursor.fetchall()

# Define table headers
headers = ["is_rented","total_listings", "avg_price","avg_price_per_sqft"]

# Print the result as a table
print(tabulate(result5, headers=headers, tablefmt="grid"))

+-------------+------------------+-------------+----------------------+
|   is_rented |   total_listings |   avg_price |   avg_price_per_sqft |
+=============+==================+=============+======================+
|           0 |            10622 | 2.44825e+06 |              776.812 |
+-------------+------------------+-------------+----------------------+
|           1 |            10578 | 2.45718e+06 |              789.364 |
+-------------+------------------+-------------+----------------------+


In [23]:
#How do bedrooms and bathrooms affect pricing?
query6 = """
SELECT
    p.bedrooms ,
    p.bathrooms,
    COUNT(*) AS total_listings,
    AVG(l.price) AS avg_price,
    AVG(l.price / l.sqft) AS avg_price_per_sqft
FROM listings l
JOIN property_attributes p
    ON l.listing_id = p.listing_id
WHERE l.sqft > 0
GROUP BY p.bedrooms, p.bathrooms 
order by p.bedrooms, p.bathrooms;
"""

cursor.execute(query6)
result6 = cursor.fetchall()

# Define table headers
    
headers = ["bedrooms","bathrooms","total_listings", "avg_price","avg_price_per_sqft"]

# Print the result as a table
print(tabulate(result6, headers=headers, tablefmt="grid"))

+------------+-------------+------------------+-------------+----------------------+
|   bedrooms |   bathrooms |   total_listings |   avg_price |   avg_price_per_sqft |
+============+=============+==================+=============+======================+
|          1 |           1 |             1040 | 2.434e+06   |              742.203 |
+------------+-------------+------------------+-------------+----------------------+
|          1 |           2 |             1047 | 2.50947e+06 |              842.036 |
+------------+-------------+------------------+-------------+----------------------+
|          1 |           3 |             1028 | 2.45903e+06 |              799.52  |
+------------+-------------+------------------+-------------+----------------------+
|          1 |           4 |             1094 | 2.45553e+06 |              752.024 |
+------------+-------------+------------------+-------------+----------------------+
|          2 |           1 |             1068 | 2.45574e+06 |    

In [24]:
#Do parking & power backup increase prices?
query7 = """
SELECT
    p.parking_available ,
    p.power_backup,
    COUNT(*) AS total_listings,
    ROUND(AVG(l.price),2) AS avg_price,
    AVG(l.price / l.sqft) AS avg_price_per_sqft
FROM listings l
JOIN property_attributes p
    ON l.listing_id = p.listing_id
WHERE l.sqft > 0
GROUP BY p.parking_available, p.power_backup 
order by avg_price_per_sqft desc;
"""

cursor.execute(query7)
result7 = cursor.fetchall()

# Define table headers
    
headers = ["parking_available","power_backup","total_listings", "avg_price","avg_price_per_sqft"]

# Print the result as a table
print(tabulate(result7, headers=headers, tablefmt="grid"))

+---------------------+----------------+------------------+-------------+----------------------+
|   parking_available |   power_backup |   total_listings |   avg_price |   avg_price_per_sqft |
+=====================+================+==================+=============+======================+
|                   1 |              1 |             5243 | 2.46565e+06 |              806.354 |
+---------------------+----------------+------------------+-------------+----------------------+
|                   0 |              0 |             5408 | 2.43668e+06 |              777.257 |
+---------------------+----------------+------------------+-------------+----------------------+
|                   1 |              0 |             5383 | 2.46236e+06 |              776.683 |
+---------------------+----------------+------------------+-------------+----------------------+
|                   0 |              1 |             5166 | 2.44629e+06 |              772.2   |
+---------------------+-------

In [25]:
# How does year built influence listing price?
query8 = """
SELECT
    p.year_built ,
    COUNT(*) AS total_listings,
    ROUND(AVG(l.price),2) AS avg_price,
    AVG(l.price / l.sqft) AS avg_price_per_sqft
FROM listings l
JOIN property_attributes p
    ON l.listing_id = p.listing_id
WHERE l.sqft > 0
GROUP BY p.year_built 
order by avg_price_per_sqft desc;
"""

cursor.execute(query8)
result8 = cursor.fetchall()

# Define table headers
    
headers = ["year_built","total_listings", "avg_price","avg_price_per_sqft"]

# Print the result as a table
print(tabulate(result8, headers=headers, tablefmt="grid"))

+--------------+------------------+-------------+----------------------+
|   year_built |   total_listings |   avg_price |   avg_price_per_sqft |
+==============+==================+=============+======================+
|         1992 |              574 | 2.59465e+06 |              831.503 |
+--------------+------------------+-------------+----------------------+
|         1993 |              548 | 2.61175e+06 |              831.342 |
+--------------+------------------+-------------+----------------------+
|         2002 |              614 | 2.43362e+06 |              827.327 |
+--------------+------------------+-------------+----------------------+
|         1997 |              610 | 2.49311e+06 |              819.123 |
+--------------+------------------+-------------+----------------------+
|         2003 |              573 | 2.39575e+06 |              816.403 |
+--------------+------------------+-------------+----------------------+
|         1998 |              649 | 2.39816e+06 |  

In [22]:
# Which cities have the highest median property prices? 

query9 = """
WITH ranked AS (
    SELECT
        City,
        Price,
        ROW_NUMBER() OVER (PARTITION BY City ORDER BY Price) AS rn,
        COUNT(*) OVER (PARTITION BY City) AS cnt
    FROM listings
)
SELECT
    City,
    ROUND(AVG(Price), 2) AS median_price
FROM ranked
WHERE rn IN ((cnt + 1) / 2, (cnt + 2) / 2)
GROUP BY City
ORDER BY median_price DESC; 
"""

cursor.execute(query9)
result9 = cursor.fetchall()

# Define table headers
    
headers = ["City","median_price"]

# Print the result as a table
print(tabulate(result9, headers=headers, tablefmt="grid"))

+-------------+----------------+
| City        |   median_price |
+=============+================+
| New York    |    2.44652e+06 |
+-------------+----------------+
| Phoenix     |    2.3976e+06  |
+-------------+----------------+
| Los Angeles |    2.38093e+06 |
+-------------+----------------+
| Chicago     |    2.35108e+06 |
+-------------+----------------+
| Houston     |    2.34758e+06 |
+-------------+----------------+


In [26]:
# How are properties distributed across price buckets?

query10 = """
SELECT
    CASE
        WHEN Price < 600000 THEN 'Below 600K'
        WHEN Price < 1100000 THEN '600K – 1.1M'
        WHEN Price < 1550000 THEN '1.1M – 1.55M'
        ELSE 'Above 1.55M'
    END AS price_bucket,
    COUNT(*) AS listings
FROM listings
GROUP BY price_bucket
ORDER BY 
    CASE price_bucket
        WHEN 'Below 600K' THEN 1
        WHEN '600K – 1.1M' THEN 2
        WHEN '1.1M – 1.55M' THEN 3
        ELSE 4
    END;
"""
cursor.execute(query10)
result10 = cursor.fetchall()

# Define table headers
    
headers = ["price_bucket","total_listings"]

# Print the result as a table
print(tabulate(result10, headers=headers, tablefmt="grid"))

+----------------+------------------+
| price_bucket   |   total_listings |
+================+==================+
| Below 600K     |             2343 |
+----------------+------------------+
| 600K – 1.1M    |             2389 |
+----------------+------------------+
| 1.1M – 1.55M   |             2199 |
+----------------+------------------+
| Above 1.55M    |            14269 |
+----------------+------------------+


In [37]:
#What is the average days on market by city?

query11 = """
SELECT
    l.City,
    avg(Days_on_Market) as average_days_on_market
FROM Sales s
INNER JOIN listings l
ON s.Listing_Id = l.Listing_Id
group by l.City
;   
"""

cursor.execute(query11)
result11 = cursor.fetchall()

# Define table headers
    
headers = ["City","average_days_on_market"]

# Print the result as a table
print(tabulate(result11, headers=headers, tablefmt="grid"))

+-------------+--------------------------+
| City        |   average_days_on_market |
+=============+==========================+
| Chicago     |                  64.2506 |
+-------------+--------------------------+
| Houston     |                  58.5111 |
+-------------+--------------------------+
| Los Angeles |                  65.1264 |
+-------------+--------------------------+
| New York    |                  60.8476 |
+-------------+--------------------------+
| Phoenix     |                  59.6543 |
+-------------+--------------------------+


In [48]:
#Which property types sell the fastest?

query12 = """
SELECT
    l.Property_Type,
    avg(Days_on_Market) as average_days_on_market
FROM Sales s
INNER JOIN listings l
ON s.Listing_Id = l.Listing_Id
group by l.Property_Type
order by average_days_on_market desc
;   
"""

cursor.execute(query12)
result12 = cursor.fetchall()

# Define table headers
    
headers = ["Property_Type","average_days_on_market"]

# Print the result as a table
print(tabulate(result12, headers=headers, tablefmt="grid"))

+-----------------+--------------------------+
| Property_Type   |   average_days_on_market |
+=================+==========================+
| Condo           |                  66.541  |
+-----------------+--------------------------+
| Townhouse       |                  60.9638 |
+-----------------+--------------------------+
| Apartment       |                  60.6475 |
+-----------------+--------------------------+
| House           |                  58.3379 |
+-----------------+--------------------------+


In [ ]:
#What percentage of properties are sold above listing price?

query13 = """
SELECT 
    (COUNT(CASE WHEN s.Sale_Price > l.Price THEN 1 END) * 100.0) / COUNT(*) 
        AS percent_sold_above_listing
FROM listings l
JOIN sales s
    ON l.Listing_ID = s.Listing_ID;
"""
cursor.execute(query13)
result13 = cursor.fetchall()

# Define table headers
    
headers = ["percent_sold_above_listing"]

# Print the result as a table
print(tabulate(result13, headers=headers, tablefmt="grid"))

+------------------------------+
|   percent_sold_above_listing |
+==============================+
|                      49.3056 |
+------------------------------+


In [50]:

#What is the sale-to-list price ratio by city?

query14 = """
SELECT 
    l.City,
    AVG(s.Sale_Price / l.Price) AS sale_to_list_ratio
FROM listings l
JOIN sales s
    ON l.Listing_ID = s.Listing_ID
GROUP BY l.City;
  
"""

cursor.execute(query14)
result14 = cursor.fetchall()

# Define table headers
    
headers = ["City","sale_to_list_ratio"]

# Print the result as a table
print(tabulate(result14, headers=headers, tablefmt="grid"))

+-------------+----------------------+
| City        |   sale_to_list_ratio |
+=============+======================+
| Chicago     |             1.00153  |
+-------------+----------------------+
| Houston     |             1        |
+-------------+----------------------+
| Los Angeles |             0.999836 |
+-------------+----------------------+
| New York    |             0.999583 |
+-------------+----------------------+
| Phoenix     |             0.998902 |
+-------------+----------------------+


In [51]:
#Which listings took more than 90 days to sell? 


query15 = """
SELECT 
    l.Listing_ID,
    l.City,
    l.Property_Type,
    s.Days_on_Market
FROM listings l
JOIN sales s
    ON l.Listing_ID = s.Listing_ID
WHERE s.Days_on_Market > 90;
  
"""

cursor.execute(query15)
result15 = cursor.fetchall()

# Define table headers
    
headers = ["Listing_ID","City","Property_Type","Days_on_Market"]

# Print the result as a table
print(tabulate(result15, headers=headers, tablefmt="grid"))

+--------------+-------------+-----------------+------------------+
| Listing_ID   | City        | Property_Type   |   Days_on_Market |
+==============+=============+=================+==================+
| L00059       | Chicago     | Townhouse       |         116      |
+--------------+-------------+-----------------+------------------+
| L00412       | Houston     | House           |         106.991  |
+--------------+-------------+-----------------+------------------+
| L00045       | Los Angeles | House           |          90.9924 |
+--------------+-------------+-----------------+------------------+
| L00241       | Phoenix     | Condo           |          92.0204 |
+--------------+-------------+-----------------+------------------+
| L00381       | Phoenix     | Apartment       |         113.007  |
+--------------+-------------+-----------------+------------------+
| L00110       | Chicago     | Condo           |         114.002  |
+--------------+-------------+-----------------+

In [56]:
#How does metro distance affect time on market?


query16 = """
SELECT 
    p.metro_distance_km,
    AVG(s.Days_on_Market) AS avg_days_on_market
FROM property_attributes p
JOIN sales s
    ON p.Listing_ID = s.Listing_ID
GROUP BY p.metro_distance_km
ORDER BY p.metro_distance_km
;  
"""

cursor.execute(query16)
result16 = cursor.fetchall()

# Define table headers
    
headers = ["metro_distance_km","avg_days_on_market"]

# Print the result as a table
print(tabulate(result16, headers=headers, tablefmt="grid"))

+---------------------+----------------------+
|   metro_distance_km |   avg_days_on_market |
+=====================+======================+
|                0.21 |            108.002   |
+---------------------+----------------------+
|                0.22 |             81.0077  |
+---------------------+----------------------+
|                0.24 |             21.9941  |
+---------------------+----------------------+
|                0.25 |             57.9996  |
+---------------------+----------------------+
|                0.26 |             99.0057  |
+---------------------+----------------------+
|                0.27 |             25.0075  |
+---------------------+----------------------+
|                0.29 |            110.996   |
+---------------------+----------------------+
|                0.31 |             33.3271  |
+---------------------+----------------------+
|                0.33 |             51.002   |
+---------------------+----------------------+
|            

In [59]:
#What is the monthly sales trend?

query17 = """
SELECT 
    strftime('%Y-%m', Date_Sold) AS sale_month,
    COUNT(*) AS total_sales
FROM sales
GROUP BY sale_month
ORDER BY sale_month;

"""

cursor.execute(query17)
result17 = cursor.fetchall()

# Define table headers
    
headers = ["sale_month","total_sales"]

# Print the result as a table
print(tabulate(result17, headers=headers, tablefmt="grid"))

+--------------+---------------+
| sale_month   |   total_sales |
+==============+===============+
| 2023-01      |             7 |
+--------------+---------------+
| 2023-02      |            17 |
+--------------+---------------+
| 2023-03      |            20 |
+--------------+---------------+
| 2023-04      |            36 |
+--------------+---------------+
| 2023-05      |            45 |
+--------------+---------------+
| 2023-06      |            31 |
+--------------+---------------+
| 2023-07      |            48 |
+--------------+---------------+
| 2023-08      |            50 |
+--------------+---------------+
| 2023-09      |            46 |
+--------------+---------------+
| 2023-10      |            64 |
+--------------+---------------+
| 2023-11      |            46 |
+--------------+---------------+
| 2023-12      |            42 |
+--------------+---------------+
| 2024-01      |            44 |
+--------------+---------------+
| 2024-02      |            44 |
+---------

In [62]:
#Which properties are currently unsold?

query18 ="""
SELECT 
    l.Listing_ID,
    l.City,
    l.Property_Type,
    l.Price
FROM listings l
LEFT JOIN sales s
    ON l.Listing_ID = s.Listing_ID
WHERE s.Listing_ID IS NULL; 

"""
cursor.execute(query18)
result18 = cursor.fetchall()

# Define table headers
    
headers = ["Listing_ID","City","Property_Type","Price"]

# Print the result as a table
print(tabulate(result18, headers=headers, tablefmt="grid"))

+--------------+-------------+-----------------+------------------+
| Listing_ID   | City        | Property_Type   |            Price |
+==============+=============+=================+==================+
| L00002       | Los Angeles | Apartment       |      1.51914e+06 |
+--------------+-------------+-----------------+------------------+
| L00005       | Phoenix     | Townhouse       | 562297           |
+--------------+-------------+-----------------+------------------+
| L00009       | New York    | Townhouse       |      1.22455e+06 |
+--------------+-------------+-----------------+------------------+
| L00014       | Chicago     | Apartment       |      1.37744e+06 |
+--------------+-------------+-----------------+------------------+
| L00015       | Los Angeles | House           |      1.06943e+06 |
+--------------+-------------+-----------------+------------------+
| L00016       | Phoenix     | House           |      1.53574e+06 |
+--------------+-------------+-----------------+

In [63]:
# Which agents have closed the most sales?

query19 = """
SELECT 
    a.Agent_ID,
    a.Name,
    COUNT(s.Listing_ID) AS total_sales_closed
FROM agents a
JOIN listings l
    ON a.Agent_ID = l.Agent_ID
JOIN sales s
    ON l.Listing_ID = s.Listing_ID
GROUP BY a.Agent_ID, a.Name
ORDER BY total_sales_closed DESC;
"""
cursor.execute(query19)
result19 = cursor.fetchall()

headers = ["Agent_ID", "Name", "Total_Sales_Closed"]
print(tabulate(result19, headers=headers, tablefmt="grid"))

+------------+-------------+----------------------+
| Agent_ID   | Name        |   Total_Sales_Closed |
+============+=============+======================+
| A0042      | Agent A0042 |                   25 |
+------------+-------------+----------------------+
| A0011      | Agent A0011 |                   24 |
+------------+-------------+----------------------+
| A0014      | Agent A0014 |                   21 |
+------------+-------------+----------------------+
| A0035      | Agent A0035 |                   21 |
+------------+-------------+----------------------+
| A0043      | Agent A0043 |                   20 |
+------------+-------------+----------------------+
| A0046      | Agent A0046 |                   20 |
+------------+-------------+----------------------+
| A0007      | Agent A0007 |                   19 |
+------------+-------------+----------------------+
| A0048      | Agent A0048 |                   19 |
+------------+-------------+----------------------+
| A0027     

In [ ]:
# Who are the top agents by total sales revenue?
query20 = """
SELECT 
    a.Agent_ID,
    a.Name,
    SUM(s.Sale_Price) AS total_sales_Revenue
FROM agents a
JOIN listings l
    ON a.Agent_ID = l.Agent_ID
JOIN sales s
    ON l.Listing_ID = s.Listing_ID
GROUP BY a.Agent_ID, a.Name
ORDER BY total_sales_Revenue DESC;
"""
cursor.execute(query20)
result20 = cursor.fetchall()

headers = ["Agent_ID", "Name", "total_sales_Revenue"] 
print(tabulate(result20, headers=headers, tablefmt="grid"))

+------------+-------------+-----------------------+
| Agent_ID   | Name        |   total_sales_Revenue |
+============+=============+=======================+
| A0011      | Agent A0011 |           2.78823e+07 |
+------------+-------------+-----------------------+
| A0042      | Agent A0042 |           2.71916e+07 |
+------------+-------------+-----------------------+
| A0043      | Agent A0043 |           2.41024e+07 |
+------------+-------------+-----------------------+
| A0035      | Agent A0035 |           2.27258e+07 |
+------------+-------------+-----------------------+
| A0014      | Agent A0014 |           2.2034e+07  |
+------------+-------------+-----------------------+
| A0046      | Agent A0046 |           2.13358e+07 |
+------------+-------------+-----------------------+
| A0048      | Agent A0048 |           2.11863e+07 |
+------------+-------------+-----------------------+
| A0027      | Agent A0027 |           2.10997e+07 |
+------------+-------------+------------------

In [68]:
#Which agents close deals fastest?

query21 = """
SELECT 
    Agent_ID,
    Name,
    avg_closing_days
FROM agents
ORDER BY avg_closing_days ASC;
"""
cursor.execute(query21)
result21 = cursor.fetchall()

headers = ["Agent_ID", "Name", "Avg_Closing_Days"]
print(tabulate(result21, headers=headers, tablefmt="grid"))

+------------+-------------+--------------------+
| Agent_ID   | Name        |   Avg_Closing_Days |
+============+=============+====================+
| A0048      | Agent A0048 |                 15 |
+------------+-------------+--------------------+
| A0035      | Agent A0035 |                 20 |
+------------+-------------+--------------------+
| A0042      | Agent A0042 |                 21 |
+------------+-------------+--------------------+
| A0023      | Agent A0023 |                 23 |
+------------+-------------+--------------------+
| A0045      | Agent A0045 |                 26 |
+------------+-------------+--------------------+
| A0041      | Agent A0041 |                 27 |
+------------+-------------+--------------------+
| A0016      | Agent A0016 |                 31 |
+------------+-------------+--------------------+
| A0034      | Agent A0034 |                 31 |
+------------+-------------+--------------------+
| A0028      | Agent A0028 |                 33 |


In [ ]:
# Does experience correlate with deals closed?

query22 = """
SELECT 
    experience_years,
    AVG(deals_closed) AS avg_deals_closed
FROM agents
GROUP BY experience_years
ORDER BY experience_years;
"""
cursor.execute(query22)
result22 = cursor.fetchall()

headers = ["Experience_Years", "Avg_Deals_Closed"]
print(tabulate(result22, headers=headers, tablefmt="grid"))

+--------------------+--------------------+
|   Experience_Years |   Avg_Deals_Closed |
+====================+====================+
|                  1 |           262      |
+--------------------+--------------------+
|                  2 |            50      |
+--------------------+--------------------+
|                  3 |            14      |
+--------------------+--------------------+
|                  4 |           140      |
+--------------------+--------------------+
|                  5 |            88.3333 |
+--------------------+--------------------+
|                  7 |            73      |
+--------------------+--------------------+
|                  9 |           283      |
+--------------------+--------------------+
|                 10 |            93.5    |
+--------------------+--------------------+
|                 11 |           109      |
+--------------------+--------------------+
|                 12 |           257      |
+--------------------+----------

In [69]:
#Do agents with higher ratings close deals faster?

query23 = """
SELECT 
    Agent_ID,
    Name,
    rating,
    avg_closing_days
FROM agents
ORDER BY rating DESC, avg_closing_days ASC;
"""
cursor.execute(query23)
result23 = cursor.fetchall()

# Define table headers
headers = ["Agent_ID", "Name", "Rating", "Avg_Closing_Days"]

# Print the result as a table
print(tabulate(result23, headers=headers, tablefmt="grid"))

+------------+-------------+----------+--------------------+
| Agent_ID   | Name        |   Rating |   Avg_Closing_Days |
+============+=============+==========+====================+
| A0026      | Agent A0026 |      5   |                 42 |
+------------+-------------+----------+--------------------+
| A0020      | Agent A0020 |      5   |                 65 |
+------------+-------------+----------+--------------------+
| A0006      | Agent A0006 |      4.9 |                 49 |
+------------+-------------+----------+--------------------+
| A0011      | Agent A0011 |      4.9 |                 60 |
+------------+-------------+----------+--------------------+
| A0016      | Agent A0016 |      4.8 |                 31 |
+------------+-------------+----------+--------------------+
| A0040      | Agent A0040 |      4.8 |                 52 |
+------------+-------------+----------+--------------------+
| A0029      | Agent A0029 |      4.8 |                 56 |
+------------+----------

In [73]:
#What is the average commission earned by each agent?
query24 = """
SELECT 
    a.Agent_ID,
    a.Name,
    AVG(a.commission_rate * s.Sale_Price) as Avg_Commission_Earned
FROM agents a
JOIN listings l
    ON a.Agent_ID = l.Agent_ID
JOIN sales s
    ON l.Listing_ID = s.Listing_ID
GROUP BY a.Agent_ID, a.Name;
"""
cursor.execute(query24)
result24 = cursor.fetchall()

# Define table headers
headers = ["Agent_ID", "Name", "Avg_Commission_Earned"]

# Print the result as a table
print(tabulate(result24, headers=headers, tablefmt="grid"))

+------------+-------------+-------------------------+
| Agent_ID   | Name        |   Avg_Commission_Earned |
+============+=============+=========================+
| A0001      | Agent A0001 |             2.42319e+06 |
+------------+-------------+-------------------------+
| A0002      | Agent A0002 |             3.23262e+06 |
+------------+-------------+-------------------------+
| A0003      | Agent A0003 |             2.77607e+06 |
+------------+-------------+-------------------------+
| A0004      | Agent A0004 |             1.93641e+06 |
+------------+-------------+-------------------------+
| A0005      | Agent A0005 |             1.1227e+06  |
+------------+-------------+-------------------------+
| A0006      | Agent A0006 |             1.74045e+06 |
+------------+-------------+-------------------------+
| A0007      | Agent A0007 |             1.72515e+06 |
+------------+-------------+-------------------------+
| A0008      | Agent A0008 |             1.73124e+06 |
+---------

In [74]:
# Which agents currently have the most active listings?

query25 = """
SELECT 
    a.Agent_ID,
    a.Name,
    COUNT(l.Listing_ID) AS active_listings
FROM agents a
JOIN listings l
    ON a.Agent_ID = l.Agent_ID
LEFT JOIN sales s
    ON l.Listing_ID = s.Listing_ID
WHERE s.Listing_ID IS NULL
GROUP BY a.Agent_ID, a.Name
ORDER BY active_listings DESC;
"""
cursor.execute(query25)
result25 = cursor.fetchall()

# Define table headers
headers = ["Agent_ID", "Name", "Active_Listings"]

# Print the result as a table
print(tabulate(result25, headers=headers, tablefmt="grid"))

+------------+-------------+-------------------+
| Agent_ID   | Name        |   Active_Listings |
+============+=============+===================+
| A0023      | Agent A0023 |               446 |
+------------+-------------+-------------------+
| A0011      | Agent A0011 |               439 |
+------------+-------------+-------------------+
| A0008      | Agent A0008 |               438 |
+------------+-------------+-------------------+
| A0042      | Agent A0042 |               435 |
+------------+-------------+-------------------+
| A0014      | Agent A0014 |               432 |
+------------+-------------+-------------------+
| A0044      | Agent A0044 |               430 |
+------------+-------------+-------------------+
| A0020      | Agent A0020 |               426 |
+------------+-------------+-------------------+
| A0012      | Agent A0012 |               425 |
+------------+-------------+-------------------+
| A0015      | Agent A0015 |               425 |
+------------+------

In [75]:
# What percentage of buyers are investors vs end users?

query26 = """
SELECT 
    buyer_type,
    COUNT(*) * 100.0 / (SELECT COUNT(*) FROM buyers) AS percentage
FROM buyers
GROUP BY buyer_type;
"""
cursor.execute(query26)
result26 = cursor.fetchall()

headers = ["Buyer_Type", "Percentage"]
print(tabulate(result26, headers=headers, tablefmt="grid"))


+--------------+--------------+
| Buyer_Type   |   Percentage |
+==============+==============+
| End User     |         50.1 |
+--------------+--------------+
| Investor     |         49.9 |
+--------------+--------------+


In [76]:
# Which cities have the highest loan uptake rate?

query27 = """
SELECT 
    l.City,
    COUNT(CASE WHEN b.loan_taken = 1 THEN 1 END) * 100.0 / COUNT(*) 
        AS loan_uptake_rate
FROM buyers b
JOIN sales s
    ON b.sale_id = s.Listing_ID
JOIN listings l
    ON s.Listing_ID = l.Listing_ID
GROUP BY l.City
ORDER BY loan_uptake_rate DESC;
"""
cursor.execute(query27)
result27 = cursor.fetchall()

headers = ["City", "Loan_Uptake_Rate (%)"]
print(tabulate(result27, headers=headers, tablefmt="grid"))


+-------------+------------------------+
| City        |   Loan_Uptake_Rate (%) |
+=============+========================+
| Los Angeles |                50.9854 |
+-------------+------------------------+
| Chicago     |                50.5662 |
+-------------+------------------------+
| New York    |                50.2232 |
+-------------+------------------------+
| Houston     |                49.7115 |
+-------------+------------------------+
| Phoenix     |                49.4971 |
+-------------+------------------------+


In [77]:
# What is the average loan amount by buyer type?

query28 = """
SELECT 
    buyer_type,
    AVG(loan_amount) AS avg_loan_amount
FROM buyers
WHERE loan_taken = 1
GROUP BY buyer_type;
"""
cursor.execute(query28)
result28 = cursor.fetchall()

headers = ["Buyer_Type", "Avg_Loan_Amount"]
print(tabulate(result28, headers=headers, tablefmt="grid"))


+--------------+-------------------+
| Buyer_Type   |   Avg_Loan_Amount |
+==============+===================+
| End User     |       5.21031e+06 |
+--------------+-------------------+
| Investor     |       5.21155e+06 |
+--------------+-------------------+


In [78]:
# Which payment mode is most commonly used?

query29 = """
SELECT 
    payment_mode,
    COUNT(*) AS usage_count
FROM buyers
GROUP BY payment_mode
ORDER BY usage_count DESC;
"""
cursor.execute(query29)
result29 = cursor.fetchall()

headers = ["Payment_Mode", "Usage_Count"]
print(tabulate(result29, headers=headers, tablefmt="grid"))


+----------------+---------------+
| Payment_Mode   |   Usage_Count |
+================+===============+
| Cash           |          5088 |
+----------------+---------------+
| UPI            |          5012 |
+----------------+---------------+
| Cheque         |          4951 |
+----------------+---------------+
| Bank Transfer  |          4949 |
+----------------+---------------+


In [79]:
# Do loan-backed purchases take longer to close?

query30 = """
SELECT 
    b.loan_taken,
    AVG(s.Days_on_Market) AS avg_days_on_market
FROM buyers b
JOIN sales s
    ON b.sale_id = s.Listing_ID
GROUP BY b.loan_taken;
"""
cursor.execute(query30)
result30 = cursor.fetchall()

headers = ["Loan_Taken (0=No,1=Yes)", "Avg_Days_On_Market"]
print(tabulate(result30, headers=headers, tablefmt="grid"))


+---------------------------+----------------------+
|   Loan_Taken (0=No,1=Yes) |   Avg_Days_On_Market |
+===========================+======================+
|                         0 |              61.071  |
+---------------------------+----------------------+
|                         1 |              62.1466 |
+---------------------------+----------------------+
